In [3]:
#Set up
import seaborn as sns
import pandas as pd
from datetime import datetime
import ast
import os
import noteql
import matplotlib.pyplot as plt

# Organisation-Identifier Networking

> Organisations publishing IATI data declare participating organisations, which have various roles in their activities. By using Organisation-Identifiers, participating organisations can be uniquely identified, allowing us to create a network.

> This application visualises the network of shared organisation references in IATI data, based on the participating-org/@ref element. You can create your own visualisation based on type of reporting organisation, specific countries, or using a list of specific organisations. 

Example network:

In [4]:
from IPython.core.display import HTML
HTML('<div class="flourish-embed flourish-chord" data-src="visualisation/17570004" data-width="29%"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

<hr>

## Running the application:

1. Select the type of visualisation you want to create from the dropdown menu: A. Reporting organisation type, B. Country, or C. A list of reporting organisations.

In [5]:
vis_type = 'Organisations by Type'

2A. If you are interested in specific organisation types, select them using the dropdown menu. Selecting a large amount of organisation types may cause this application to run slowly or fail.

In [6]:
rep_org_type = ['10']

**Organisation Types:**

**Code** | **Name**     |
|---------|---------------------------------|
| 10 | Government |
| 11 | Local Government |
| 15 | Other Public Sector |
| 21 | International NGO |
| 22 | National NGO |
| 23 | Regional NGO |
| 24 | Partner Country based NGO | 
| 30 | Public Private Partnership |	
| 40 | Multilateral |
| 60 | Foundation |
| 70 | Private Sector |
| 71 | Private Sector in Provider Country |
| 72 | Private Sector in Aid Recipient Country |
| 73 | Private Sector in Third Country |
| 80 | Academic, Training and Research | 	
| 90 | Other 


Get info country selector dropdown.

In [7]:
#download latest registry info
!wget https://iatiregistry.org/publisher/download/csv
registry = pd.read_csv("csv")
!rm -rf "csv" 

countries = sorted(registry['HQ Country or Region'].unique())

2B.  If you are interested in reporting organisations based in a specific country or countries, select them using the dropdown menu. You can find out where an organisation is based using the registry. Selecting a large number of countries may cause this application to run slowly or fail.

In [8]:
country_query = ['(No country assigned)']

2C.  If you are interested in a list of specific reporting organisations, enter the Organisation Identifiers of the IATI Publishers you want to select into the text box, copying this formatting exactly: 'AU-5', '44000'.  You can find these identifiers using the registry.

In [9]:
pub_org_id = ''

Reporting organisation identifiers:

In [10]:
registry[['Publisher', 'IATI Organisation Identifier','HQ Country or Region']]

3. Click the "Run" button.

<button>Run</button>

In [11]:
if vis_type in ['Organisations by Type']:
    print(f"Last run on organisation types:",*rep_org_type)
elif vis_type in ['Country']:
    print(f"Last run on organisations with headquarters in:",*country_query)
elif vis_type in ['Specific Organisations']:
    print(f"Last run on reporting organisations:",*pub_org_id)

# Download data

## Setup

Using https://iati-tables.opendata.coop/. The following cell sets up the basic libraries needed for analysis and makes a connection to the database. 

In [9]:
sns.set_context('notebook')

#start noteql session
session = noteql.local_db_session()
# Restart postgres to make sure any existing connections get dropped
!sudo service postgresql restart
session = noteql.Session(datasette_url='https://datasette.codeforiati.org/iati.json', connect_args={'connect_timeout': 1000})

## IATI-tables Queries

Downloading all participating organisations reported by selected reporting org type, countries, or ids.

Prepare country filter - based on ISO code as much as possible.

In [10]:
#find country iso codes
iso_codes = pd.read_csv("Country.csv")
iso_codes = iso_codes[iso_codes['name'].isin(country_query)]
iso_codes['code']

In [11]:
#filter reporting orgs to countries of interest
#should I include filtering to org-id with prefix too??
filt_reg = registry[registry['HQ Country or Region'].isin(country_query)]
#split iso codes from IDs
split_id = filt_reg['IATI Organisation Identifier'].str.split('-',n=1, expand=True)
country_orgs = pd.concat([filt_reg , split_id], axis=1)
country_orgs

Format input text

In [12]:
pub_orgs = ast.literal_eval(pub_org_id)
pub_orgs = [n.strip() for n in pub_orgs] 
pub_orgs

In [13]:
outputs = {}

if vis_type in ['Organisations by Type']:
    for i in rep_org_type:
        #tables query 
        df = %nql SELECT DISTINCT a.prefix, a.reportingorg_ref, b.reportingorg_type, b.reportingorg_typename, a.ref, a.type, a.typename, a.narrative FROM participatingorg as a LEFT JOIN activity as b on (b._link = a._link_activity) WHERE (b.reportingorg_type = {{i}}) ORDER BY a.narrative
        outputs[i] = df
elif vis_type in ['Country']:
    for i in country_orgs['IATI Organisation Identifier']:
        #tables query 
        df = %nql SELECT DISTINCT a.prefix, a.reportingorg_ref, a.ref, a.type, a.typename, a.narrative FROM participatingorg as a WHERE (a.reportingorg_ref = {{i}}) 
        outputs[i] = df
elif vis_type in ['Specific Organisations']:
    for i in pub_org_id :
        #tables query 
        df = %nql SELECT DISTINCT a.prefix, a.reportingorg_ref, a.ref, a.type, a.typename, a.narrative FROM participatingorg as a WHERE (a.reportingorg_ref = {{i}})
        outputs[i] = df

In [14]:
part_orgs =  pd.concat(outputs.values(), ignore_index=True)
part_orgs

In [15]:
!rm -rf part_orgs.xlsx
part_orgs.to_excel("part_orgs.xlsx",sheet_name="part_orgs", index=False)

# Clean data

## Setup

In [16]:
session2 = noteql.local_db_session()
# Restart postgres to make sure any existing connections get dropped
!sudo service postgresql restart
# We can't get the noteql session to do these, because it wraps sql in a transaction, which isn't allowed for DROP/CREATE
!sudo -u postgres psql -c "DROP DATABASE IF EXISTS pa"
!sudo -u postgres psql -c "CREATE DATABASE pa"
session2 = noteql.Session("postgresql+psycopg2://root@/pa", "public")

## Load data

In [17]:
part_orgs = pd.read_excel('part_orgs.xlsx',sheet_name=None)

for key in part_orgs.keys():
    table_name = key.replace(' ','')
    table_name = table_name.lower()
    print(table_name)
    %nql DROP TABLE IF EXISTS {{table_name | ident}}; SELECT ''
    # Convert to a DataFrame and render
    df = pd.DataFrame.from_dict(part_orgs[key])
    session2.load_dataframe(df, table_name, index=False)

In [18]:
!wget https://iatistandard.org/reference_downloads/203/codelists/downloads/clv3/csv/en/OrganisationRegistrationAgency.csv
!wget https://iatistandard.org/reference_downloads/203/codelists/downloads/clv3/csv/en/OrganisationIdentifier.csv 

reg_agency = pd.read_csv("OrganisationRegistrationAgency.csv")
old_orgid = pd.read_csv("OrganisationIdentifier.csv")
refs = pd.read_csv("Generic_XM-DACs_Jul24.csv")

session2.load_dataframe(registry, "registry", index=False)
session2.load_dataframe(reg_agency, "reg_agency", index=False)
session2.load_dataframe(old_orgid, "old_orgid", index=False)
session2.load_dataframe(refs,"refs",index=False)

!rm -rf "OrganisationRegistrationAgency.csv" "OrganisationIdentifier.csv" 

- Remove obvious issues such as blank references 

- Remove all/part of the data from reporting orgs with known issues which mean a participating organisation could not be identified. 

- Trim leading/trailing whitespace from narratives.

- Remove generic XM-DAC codes.

Other issues will be caught by the filter data section, which checks for valid organisation-ids with a format of {RegistrationAgency}-{RegistrationNumber}.

In [19]:
refs['code'] = refs['code'].astype(str) 
refs['XMDAC'] = "XM-DAC-"+refs['code']
refs

In [20]:
%%nql SHOW CREATE cleaned_part_orgs

SELECT DISTINCT
    prefix,
    reportingorg_ref,
    ref,
    type,
    typename,
    CASE
        WHEN
            SPLIT_PART(ref,'-',2) = '' THEN ''
            ELSE CONCAT(SPLIT_PART(ref,'-',1),'-',SPLIT_PART(ref,'-',2))
    END as "reg_agency",
    TRIM(REGEXP_REPLACE(narrative, E'\n', '')) as "narrative"
FROM part_orgs
WHERE NOT narrative in ('',' ','EN: ','Odefinierat','EN: IP not published','Not applicable',
                        'Confidential',
                        'USAID redacted this field in accordance with the exceptions outlined in the Foreign Aid Transparency and Accountability Act of 2016.')
    AND NOT POSITION('Private donors' IN narrative) > 0
    AND NOT POSITION('Övrigt' IN narrative) > 0
    AND NOT POSITION('Övriga' IN narrative) > 0
    AND CASE
        WHEN prefix='unicef' THEN NOT POSITION('XM-DAC-41122-VN-' IN ref) > 0 
        ELSE true
    END
    AND NOT ref in (' ','Not registered in IATI','0')
    AND NOT ref in {{ refs['code'] | inclause }}
    AND NOT ref in {{ refs['XMDAC'] | inclause }}

## Add other data sources

- Join to registry IDs, registration agency, and old IATI codelist. 

- Add name of reporting-org from registry. If the reporting org reference does not match the registry, it will be removed. 

In [21]:
%%nql SHOW CREATE joined_part_orgs 
SELECT 
    e."Publisher" as "reportingorg_name",
    a.*,
    b."Publisher" as "registry_narrative",
    c.name as "reg_agency_narrative",
    d.name as "v1_orgid"
FROM cleaned_part_orgs as a
    LEFT JOIN registry as b ON a.ref = b."IATI Organisation Identifier"
    LEFT JOIN reg_agency as c ON a.reg_agency = c.code
    LEFT JOIN old_orgid as d ON a.ref = d.code
    LEFT JOIN registry as e ON a.reportingorg_ref = e."IATI Organisation Identifier"
WHERE NOT e."Publisher" = 'None'
ORDER BY a.prefix

## Filter data

Check that the participating organisation references:

- Are a valid reporting org ID from the registry OR

- Have a valid prefix from org-id UNLESS

- They are a v1 org id

In [22]:
%%nql SHOW CREATE filtered_orgs output=DF
SELECT
    *
FROM joined_part_orgs 
WHERE NOT reg_agency_narrative = 'None'
    OR NOT registry_narrative = 'None'
    OR NOT v1_orgid = 'None'
ORDER BY prefix

In [ ]:
!rm -rf output.xlsx
output.to_excel("output.xlsx",sheet_name="output", index=False)

output_file_url = f"https://deepnote.com/publish/{os.environ['DEEPNOTE_PROJECT_ID']}/file?path=output.xlsx"
print(output_file_url)

# Summarise

<hr>

## Chord counts - to download

Code source: https://stackoverflow.com/questions/64689296/counting-shared-elements-within-groups-with-condition?rq=3 

Chord output for use with bokeh:

In [28]:
#Get unique reporting org name and participating org reference combinations
df = output[["reportingorg_name","ref"]].drop_duplicates()

result = df.merge(df, on='ref').groupby(['reportingorg_name_x','reportingorg_name_y']).count().reset_index()
result.columns = ['source', 'target', 'value']
mask = result.source!=result.target

chord_df = result[mask].reset_index(drop='True')
chord_df  = chord_df.sort_values(by="value", ascending=False)
chord_df

## Networking Visualisation

Use Bokeh to create a chord diagram

In [29]:
import holoviews as hv
import numpy as np
from holoviews import opts, dim
from bokeh.plotting import figure, output_file, save, show
from IPython.display import IFrame
from IPython.core.display import display, HTML
import tempfile

Workaround to use Bokeh in Deepnote

In [30]:
def bokeh_deepnote_show(plot):
    tmp_output_filename = tempfile.NamedTemporaryFile(suffix='.html').name
    output_file(tmp_output_filename)
    save(plot)

    f = open(tmp_output_filename, "r")
    display(HTML(f.read()))

Adjust the minimum number of shared participating organisation references by using the slider. Use the "visualise" button to refresh the diagram. Note: currently the whole application must re-run to recreate this diagram, so this may take some time.

In [31]:
filter = 0

<button>Visualise</button>

In [32]:
mask = np.logical_and(result.source<result.target, result.value>0)

bokeh_df = result[mask].reset_index(drop='True')
bokeh_df  = bokeh_df.sort_values(by="source")
bokeh_df

In [33]:
# Should the diagram be plotted with 'bokeh' or 'matplotlib'?
hv.extension('bokeh')

#filter out orgs with few in common
filtered_data = bokeh_df[bokeh_df['value'] >= filter]

#Create chord labels
orgs = list(set(filtered_data ["source"].unique().tolist() + filtered_data ["target"].unique().tolist()))
orgs_dataset = hv.Dataset(pd.DataFrame(orgs, columns=["Organisation"]))

chord = hv.Chord((filtered_data,orgs_dataset))

chord.opts(cmap = 'Category20',  
          edge_cmap = 'Category20',  
          labels = 'Organisation',
          node_color = hv.dim('Organisation').str(),
          #edge_color = hv.dim('source').str(), 
          title = 'Shared Participating Organisation References',
          width = 900,
          height = 900)

bokeh_deepnote_show(hv.render(chord))

# Output

In [34]:
!rm -rf chord_df.xlsx
chord_df.to_excel("chord_df.xlsx",sheet_name="chord_df", index=False)

chord_file_url = f"https://deepnote.com/publish/{os.environ['DEEPNOTE_PROJECT_ID']}/file?path=chord_df.xlsx"
print(chord_file_url)

<hr>

# Further visualisation

To create and edit your own diagram, you may wish to use the data visualisation tool "Flourish". 

You can download the chord data produced by this application here: Download.

Example flourish chord diagram:

In [35]:
from IPython.core.display import HTML
HTML('<div class="flourish-embed flourish-chord" data-src="visualisation/17570004" data-width="29%"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

<hr>

## How is the data cleaned?

Organisation identifiers with known issues are removed - these include redacted or blank narratives, and internal or generic references, as they cannot be used for the networking of IATI data. This is not an exhaustive list and is subject to change.  

Blank/internal/generic references

 Blank/redacted narratives

| **Organisation** | **Redaction message/Issue**     |
|---------|---------------------------------|
| iom     | Confidential                    |
| sida | Övriga/Övrigt ... (Other) |
| unicef  | EN: IP not published            |
| unhcr   | Private donors in...             |
|usaid | USAID redacted this field in accordance with the exceptions outlined in the Foreign Aid Transparency and Accountability Act of 2016.|

| **Organisation** | **Redaction message/Issue**     |
|------------------|---------------------------------|
| unicef            | Internal ID eg XM-DAC-41122-VN-242363 |
| Multiple orgs | Generic XM-DAC- codes used |

TO DO - compare DAC codes to https://github.com/pwyf/aid-transparency-tracker/blob/main/beta/excluded_xm_dac.py

Participating organisation identifiers must have a valid format of {RegistrationAgency}-{RegistrationNumber}, or match a known organisation-identifier as recorded in the registry.

You can download the cleaned data here: Download

<hr>

## Data Snapshots

Number of reporting organisations referencing each participating organisation identifier :

In [24]:
%%nql SHOW CREATE ref_count ref_count=DF

SELECT 
    ref as "Participating Organisation Identifier",
    COUNT(DISTINCT(reportingorg_ref)) as "Count" 
FROM filtered_orgs 
GROUP BY ref
ORDER BY "Count" DESC

Most commonly referenced participating organisation identifiers - Top 25

In [25]:
plt.barh("Participating Organisation Identifier", "Count", data = ref_count.loc[0:24])
plt.xlabel('Count of reporting organisations')
plt.ylabel('Participating Organistion Reference')
plt.show()

Count of unique valid participating organisation identifiers referenced by each reporting organisation:

In [26]:
%%nql SHOW CREATE reporg_count reporg_count=DF

SELECT
    reportingorg_name as "Reporting Organisation",
    COUNT(DISTINCT(ref)) as "Count of Participating Organisation Identifiers"
FROM filtered_orgs 
GROUP BY reportingorg_name
ORDER BY "Count of Participating Organisation Identifiers" DESC

Reporting organisations with most valid participating organisation identifiers - Top 25

In [27]:
plt.barh('Reporting Organisation', 'Count of Participating Organisation Identifiers', 
            data = reporg_count.loc[0:24])
plt.xlabel('Count of Participating Organisation References')
plt.ylabel('Reporting Organistion')
plt.show()

<hr>

### Footnote

The code behind this application is hidden to improve usability. You are welcome to view this code at: Notebook.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=240fcfd1-8557-41e6-8271-b13ecef554c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>